In [ ]:
import numpy as np 
import pandas as pd
import random
import matplotlib.pyplot as plt # to visualize only
from skimage.transform import resize, rescale
from sklearn.model_selection import KFold

In [ ]:
data_x_pd = pd.read_csv("./data/processed_x.csv", delimiter=",", header = None) # load from processed images
# train_x = train_x.values # dataframe to numpy ndarray
# train_x = train_x.astype(np.float32)
# train_x /= 255 # normalize

In [ ]:
data_y_pd = pd.read_csv("./data/train_y.csv", delimiter = ",", header = None)
# train_y = (train_y.values).ravel()

In [ ]:
# # Can see resized 28x28 image
# def plotter(images, labels, start):
#     plt.figure(figsize=(20,4))
#     for index, (image, label) in enumerate(zip(images[start:start+5], labels[start:start+5])):
#         plt.subplot(1, 5, index + 1)
#         im = np.reshape(image, (64,64))
#         im28 = resize(im, (28, 28), preserve_range=True)
# #         im28 = rescale(im, 0.445,preserve_range=True)
#         plt.imshow(im28, cmap= 'gray')
#         plt.title('Training: %i\n' % label, fontsize = 20)
# plotter(train_x,train_y, 0) # first 5 images

In [ ]:
def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [ ]:
class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            valid_data):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if valid_data: n_valid = len(valid_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
#             mini_batches = [
#                 training_data[k:k+mini_batch_size]
#                 for k in xrange(0, n, mini_batch_size)]
#             for mini_batch in mini_batches:
            self.update_mini_batch(training_data, eta)
            
            valid_score = self.evaluate(valid_data)
            print "Epoch {0}: {1} / {2}".format(
                j, valid_score, n_valid)
        
        return valid_score
        
    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]
#         self.weights = [w-(eta)*nw
#                         for w, nw in zip(self.weights, delta_nabla_w)]
#         self.biases = [b-(eta)*nb
#                        for b, nb in zip(self.biases, delta_nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
            
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        
#         for x, y in test_data:
#             print self.feedforward(x)
#             print y
            
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

In [ ]:
# data_x = data_x_pd.head(1000)
# data_y = data_y_pd.head(1000)

# data = pd.concat([data_x, data_y], axis=1)
# data.columns = [i for i in range(data.shape[1])]

# # test_x = train_x.tail(100)
# # test_y = train_y.tail(100)

# fifth = len(data.index)/5

# sets  = []
# sets.append([data.head(fifth).reset_index(), data.tail(len(data)-fifth).reset_index()])
# sets.append([data.iloc[fifth:fifth*2].reset_index(), data.tail(len(data)-fifth*2).append(data.head(fifth)).reset_index()])
# sets.append([data.iloc[fifth*2:fifth*3].reset_index(), data.tail(fifth*2).append(data.head(fifth*2+4)).reset_index()])
# sets.append([data.iloc[fifth*3:fifth*4].reset_index(), data.tail(fifth+4).append(data.head(fifth*3)).reset_index()])
# sets.append([data.tail(fifth).reset_index(), data.head(fifth*4+4).reset_index()])

# average_accuracy = 0
# layer_dims = [[4096,100,10],[4096,300,10],[4096,1000,10],
#               [4096,300,100,10],[4096,500,150,10]]
# learning_rates = [0.1,1.0,3.0,10.0]
# for i in range(1):
#     # Training and validation set of chunk i
#     trainX = sets[i][1][sets[i][1].columns[:-1]].drop(['index'], axis=1)
#     trainY = sets[i][1][sets[i][1].columns[-1:]]
#     trainY.columns = [0]
    
#     validX = sets[i][0][sets[i][0].columns[:-1]].drop(['index'], axis=1)
#     validY = sets[i][0][sets[i][0].columns[-1:]]
#     validY.columns = [0]
    
#     training_y = map(int, trainY[0].tolist())
#     valid_y = map(int, validY[0].tolist())

#     for i in range(len(training_y)):
#         temp = np.zeros((10,1))
#         temp[training_y[i]] = 1
#         training_y[i] = temp
    
#     training_x = trainX.values.astype(np.float32)
#     training_x /= 255

#     valid_x = validX.values.astype(np.float32)
#     valid_x /= 255

#     training_inputs = [np.reshape(x,(training_x.shape[1],1)) 
#                        for x in training_x]
#     valid_inputs = [np.reshape(x,(valid_x.shape[1],1)) for x in valid_x]
#     training_data = zip(training_inputs, training_y)
#     valid_data = zip(valid_inputs, valid_y)
    
#     print training_y
    
#     neural_net = Network([4096, 300, 10])
    
#     valid_score = neural_net.SGD(training_data, 100, 4096, 3.0, valid_data)
#     average_accuracy += valid_score

# print average_accuracy / 5.0

In [ ]:
### Prepare Data for Neural Net ###

data_x = data_x_pd.head(1000).values.astype(np.float32)
data_y = np.array(map(int, data_y_pd.head(1000)[0].tolist()), dtype=object)
# data_y = data_y_pd.head(1000).values

# resive to 28x28 image NOT DONE
# training_x_im = [np.reshape(im, (64,64)) for im in training_x]
# training_x_resize = [resize(im, (28, 28), preserve_range=True) 
#                          for im in training_x_im]
# training_x = [im.reshape(728) for im in training_x_resize]

# for i, image in enumerate(data_x):
    
# Normalize x values
data_x /= 255

kf = KFold(n_splits=5)
kf.get_n_splits(data_x)

average_score = 0
layer_dims = [[4096,100,10],[4096,300,10],[4096,1000,10],
              [4096,300,100,10],[4096,500,150,10]]
learning_rates = [0.1,1.0,3.0,10.0]

for train_index, valid_index in kf.split(data_x):
    train_x, valid_x = data_x[train_index], data_x[valid_index]
    train_y, valid_y = data_y[train_index], data_y[valid_index]
    
    # vectorize training targets
    for i in range(len(train_y)):
        vect = np.zeros((10,1), dtype=object)
        vect[train_y[i]] = 1
        train_y[i] = vect.astype(float)
    
    # reshape and zip train and valid
    training_inputs = [np.reshape(x,(train_x.shape[1],1)) 
                       for x in train_x]
    valid_inputs = [np.reshape(x,(valid_x.shape[1],1)) for x in valid_x]
    training_data = zip(training_inputs, train_y)
    valid_data = zip(valid_inputs, valid_y)

    neural_net = Network([4096, 100, 10])
    valid_score = neural_net.SGD(training_data, 100, 4096, 3.0, valid_data)
    average_accuracy += valid_score

# print average_accuracy / 5.0